# Quickstart: Building a Semantic Layer with Data-Tools

This notebook provides a quick introduction to the `data-tools` project. You'll learn how to use its key features to automatically build a semantic layer over your data.

**What is a Semantic Layer?**

A semantic layer is a business-friendly representation of your data. It hides the complexity of the underlying data sources and provides a unified view of your data using familiar business terms. This makes it easier for business users to understand and query the data without needing to be SQL experts.

**Who is this for?**

This tool is designed for both **data teams** and **business teams**. 

* **Data teams** can use it to automate data profiling, schema discovery, and documentation, significantly accelerating their workflow.
* **Business teams** can use it to gain a better understanding of their data and to perform self-service analytics without needing to write complex SQL queries.

**In this notebook, you will learn how to:**

1. **Configure your LLM Provider:** Set up the Large Language Model that will power the automated features.
2. **Profile your data:** Analyze your data sources to understand their structure, data types, and other characteristics.
3. **Automatically predict links:** Use a Large Language Model (LLM) to automatically discover relationships (foreign keys) between tables.
4. **Generate a semantic layer:** Create a `manifest.json` file that defines your semantic layer.
5. **Generate SQL queries:** Use the semantic layer to generate SQL queries and retrieve data.

**In this notebook, you will learn how to:**

1. **Profile your data:** Analyze your data sources to understand their structure, data types, and other characteristics.
2. **Automatically predict links:** Use a Large Language Model (LLM) to automatically discover relationships (foreign keys) between tables.
3. **Generate a semantic layer:** Create a `manifest.json` file that defines your semantic layer.
4. **Generate SQL queries:** Use the semantic layer to generate SQL queries and retrieve data.

## 1. LLM Configuration

Before running the project, you need to configure a Large Language Model (LLM). This is used for tasks like generating business glossaries and predicting links between tables.

You can configure the LLM by setting the following environment variables:

* `LLM_PROVIDER`: The LLM provider and model to use (e.g., `openai:gpt-3.5-turbo`).
* `OPENAI_API_KEY`: Your API key for the LLM provider.

Here's an example of how to set these variables in your environment:

```bash
export LLM_PROVIDER="openai:gpt-3.5-turbo"
export OPENAI_API_KEY="your-openai-api-key"
```

Alternatively, you can set them in the notebook like this:

In [1]:
import os

os.environ["LLM_PROVIDER"] = "openai:gpt-3.5-turbo"
os.environ["OPENAI_API_KEY"] = "your-openai-api-key" # Replace with your actual key

## 2. Data Profiling

The first step in building a semantic layer is to profile your data. This involves analyzing your data sources to understand their structure, data types, and other characteristics. The `data-tools` project provides a pipeline for this purpose.


In [2]:
import pandas as pd

from data_tools import DataSet

/home/raphael/intugle/data-tools/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/raphael/intugle/data-tools/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "schema" in "Source" shadows an attribute in parent "BaseResource"
  warnings.warn(


In [3]:
# Load the sample data
allergies_df = pd.read_csv('https://raw.githubusercontent.com/Intugle/data-tools/refs/heads/main/sample_data/healthcare/allergies.csv')
allergies_df.head()

,start,stop,patient,encounter,code,system,description,type,category,reaction1,description1,severity1,reaction2,description2,severity2
0,2020-02-17,NaN,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,01efcc52-15d6-51e9-faa2-bee069fcbe44,111088007,Unknown,Latex (substance),allergy,environment,247472004.0,Wheal (finding),MILD,NaN,NaN,NaN
1,2020-02-17,NaN,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,01efcc52-15d6-51e9-faa2-bee069fcbe44,84489001,Unknown,Mold (organism),allergy,environment,76067001.0,Sneezing,MILD,NaN,NaN,NaN
2,2020-02-17,NaN,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,01efcc52-15d6-51e9-faa2-bee069fcbe44,260147004,Unknown,House dust mite (organism),allergy,environment,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-17,NaN,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,01efcc52-15d6-51e9-faa2-bee069fcbe44,264287008,Unknown,Animal dander (substance),allergy,environment,878820003.0,Rhinoconjunctivitis (disorder),MODERATE,271807003.0,Eruption of skin (disorder),MILD
4,2020-02-17,NaN,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,01efcc52-15d6-51e9-faa2-bee069fcbe44,256277009,Unknown,Grass pollen (substance),allergy,environment,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Create a DataSet object and run the profiling pipeline
dataset_allergies = DataSet(allergies_df, "allergies")
dataset_allergies.run(domain="Healthcare")

/home/raphael/intugle/data-tools/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/raphael/intugle/data-tools/.venv/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
[!] Empty column encountered for allergies ==> stop ...
  0%|          | 0/15 [00:00<?, ?it/s][!] Error while llm invoke: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-ope*******-key. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
[!] Error while parsing: list index out of range
 13%|█▎        | 2/15 [00:00<00:02,  6.21it/s][!] Error while llm invoke: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-ope*******-key. You can find your API key at https://platf

ValidationError: 1 validation error for ColumnGlossary
business_tags
  Input should be a valid list [type=list_type, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type

The `run()` method performs a series of analysis steps, including:

* **Profiling:** Calculates statistics for each column, such as distinct count, uniqueness, and completeness.
* **Datatype Identification:** Identifies the data type of each column (e.g., integer, string, datetime).
* **Key Identification:** Identifies potential primary keys.
* **Glossary Generation:** Generates a business glossary for each column using an LLM.

> The `domain` parameter helps the LLM generate a more contextual business glossary. It specifies the industry domain that the dataset belongs to (e.g., "Healthcare", "Finance", "E-commerce").


## 3. Automated Link Prediction

Now that we've profiled our data, let's discover the relationships between tables. The `data-tools` project uses a LLM to predict links (foreign keys) between tables.

First, we'll load a few more tables from the sample dataset.

In [ ]:
table_names = ["patients", "claims", "careplans", "claims_transactions", "medications"]
datasets = [dataset_allergies]


def generate_table_url(table_name: str) -> str:
    """Append the base URL to the table name."""
    return f"https://raw.githubusercontent.com/Intugle/data-tools/refs/heads/main/sample_data/healthcare/{table_name}.csv"


for table_name in table_names:
    table_url = generate_table_url(table_name)
    df = pd.read_csv(table_url)
    dataset = DataSet(df, table_name)
    dataset.run(domain="Healthcare")
    datasets.append(dataset)

Now, let's run the link prediction pipeline.

In [ ]:
from data_tools import LinkPredictor

# Initialize the predictor
predictor = LinkPredictor(datasets)

# Run the prediction
results = predictor.predict()
results.links

The `results` object contains the predicted links between the tables. You can also visualize the relationships as a graph.


In [ ]:
results.show_graph()


## 4. The Semantic Layer (Manifest)

The profiling and link prediction results are used to generate a `manifest.json` file. This file defines the semantic layer, including the models (tables) and their relationships.

Let's save the datasets and the predicted links to YAML files. By default, these files are saved in a `sources` directory in the current working directory. You can configure this path by setting the `PROJECT_BASE` environment variable.

In [ ]:
for ds in datasets:
    ds.save_yaml()

results.save_yaml("relationships.yml")

Now, we can load the YAML files and create a manifest.

## 5. SQL Generation

Once you have a semantic layer, you can use the `SqlGenerator` to generate SQL queries. This allows you to query the data using business-friendly terms, without having to write complex SQL.

Let's create an ETL model to define the query we want to generate.

In [ ]:

etl = {
    "name": "test_etl",
    "fields": [
        {"id": "patients.first", "name": "first_name"},
        {"id": "patients.last", "name": "last_name"},
        {"id": "allergies.start", "name": "start_date"},
        {"id": "encounters.total_claim_cost", "name": "claim_cost", "category": "measure", "measure_func": "sum"},
    ],
    "filter": {
        "selections": [{"id": "claims.departmentid", "values": ["3", "20"]}],
        "sort_by": [{"id": "encounters.start", "direction": "desc"}],
    },
}

Now, let's use the `SqlGenerator` to generate the SQL query.

In [ ]:
from data_tools.sql_generator import SqlGenerator

# Create a SqlGenerator
sql_generator = SqlGenerator()

# Generate the query
sql_query = sql_generator.generate_query(etl)

# Print the query
print(sql_query)

## Conclusion

This notebook has provided a brief overview of the `data-tools` project. You've learned how to:

* Configure your LLM provider.
* Profile your data to understand its characteristics.
* Use an LLM to automatically predict links between tables.
* Generate a semantic layer (`manifest.json`).
* Use the semantic layer to generate SQL queries.

This is just a starting point. The `data-tools` project has many other features to explore. We encourage you to try it with your own data and see how it can help you build a powerful semantic layer.
